# **Import Statements**

In [ ]:
import opensim as osim
import matplotlib.pyplot as plt
import numpy as np
from Bicep import Bicep_Curl
osim.GetVersionAndDate()


%load_ext autoreload
%autoreload 2

# **Model Parameters**

In [ ]:
# Load the model.
arm26 = osim.Model('OpenSIM_utils\\Arm26\\arm26.osim')

# Print metadata.
print("Name of the model:", arm26.getName())
print("Author:", arm26.get_credits())
print("Publications:", arm26.get_publications())
print("Length Unit:", arm26.get_length_units())
print("Force Unit:", arm26.get_force_units())
print("Gravity:", arm26.get_gravity())

# Use the TableProcessor to read the motion file.
tableTime = osim.TimeSeriesTable('OpenSIM_utils\\Arm26\\OutputReference\\InverseKinematics\\arm26_InverseKinematics.mot')
print(tableTime.getColumnLabels())

#Named variables
r_shoulder_elev = tableTime.getDependentColumn('r_shoulder_elev').to_numpy()
r_elbow_flex = tableTime.getDependentColumn('r_elbow_flex').to_numpy()
t = tableTime.getIndependentColumn()


# **Simulation**

In [ ]:
step_per_second = 100
curl_time = 2 #in seconds
bicep = Bicep_Curl(curl_time = curl_time, sps = step_per_second)

# Run Simulation Under zero external force
activation = []
for i in range(int(curl_time*step_per_second)):
    activation.append(bicep.step_simulation(i, [0, -25.0, 0]))


#Plotting
t, elbow_traj = bicep._traj()
plt.plot(t, np.rad2deg(elbow_traj)) 
plt.xlabel("Time (s)")
plt.ylabel("Joint Angle (Deg)")
plt.title("Specified Elbow Joint Exercise Trajectory")
plt.show()

activation = np.asarray(activation)*100
plt.plot(t, activation)
plt.xlabel("Time (s)")
plt.ylabel("Biceps Activation Percentage")
plt.title("Simulated Biceps Activation Trajectory")
plt.show()